In [1]:
from json import JSONDecoder


In [2]:
def read_JSON_as_list(filename):
    def parse_pairs(pairs):
        return pairs

    decoder = JSONDecoder(object_pairs_hook=parse_pairs)
    with open(filename) as json_file:
        file_content = json_file.read()
        data = decoder.decode(file_content)
        
    return data

In [3]:
data = read_JSON_as_list('./JSON/mental_imagery_extended.json')

# Get all object instances

In [4]:
# data[1][1] has the value for 'steps' --> a list of the different objects 
# (except for the ones in groups)
objects = data[1][1]

# e.g. data[1][1][0] is a NeuralField, its first element is the type of object,
# i.e. in this case the string 'cedar.dynamics.NeuralField', its second element
# contains the parameters and their values
print(objects[0][0], "\n")

print(objects[0][1], "\n")

# for obj in objects:
#     print(obj[0])

cedar.dynamics.NeuralField 

[('name', 'Above Memory'), ('activation as output', 'false'), ('discrete metric (workaround)', 'false'), ('update stepIcon according to output', 'true'), ('threshold for updating the stepIcon', '0.80000000000000004'), ('dimensionality', '0'), ('sizes', ''), ('time scale', '100'), ('resting level', '-5'), ('input noise gain', '0.10000000000000001'), ('sigmoid', [('type', 'cedar.aux.math.AbsSigmoid'), ('threshold', '0'), ('beta', '100')]), ('global inhibition', '-0.01'), ('lateral kernels', [('cedar.aux.kernel.Box', [('dimensionality', '1'), ('anchor', ['0']), ('amplitude', '6.0999999999999996'), ('widths', ['2'])])]), ('lateral kernel convolution', [('borderType', 'Zero'), ('mode', 'Same'), ('engine', [('type', 'cedar.aux.conv.OpenCV')]), ('alternate even kernel center', 'false')]), ('noise correlation kernel', [('dimensionality', '1'), ('anchor', ['0']), ('amplitude', '0'), ('sigmas', ['3']), ('normalize', 'true'), ('shifts', ['0']), ('limit', '5')]), ('com

In [5]:
# get all objects that should be created, also the ones in groups
# get dictionary which translates keys to positions
key_to_pos_dict = {}
for ind, elem in enumerate(data):
    key_to_pos_dict[elem[0]] = ind

# list of the groups, each group's first element is its name, 
# second element is the group value
groups = data[key_to_pos_dict['groups']][1]

# no sure if this is needed later
group_key_to_pos = {}

for ind, group in enumerate(groups):
    group_key_to_pos[group[0]] = ind
    # group[1] --> get value of this group, i.e. everything except the name of the group
    # group[1][1] --> of this value get the value that is at position 1, i.e. in 'steps'
    # group[1][1][1] --> get the values that are in steps
    for obj in group[1][1][1]:
        # add the object to the list of objects
        objects.append(obj)

In [6]:
# check if I have all objects now by checking if the numbers are the same as
# in the json file
object_counts = {}

for obj in objects:
    if obj[0] not in object_counts.keys():
        object_counts[obj[0]] = 1
    else:
        object_counts[obj[0]] += 1
        
for key in object_counts:
    print('Number of %s: %i' %(key, object_counts[key]))

# --> Numbers check out

Number of cedar.dynamics.NeuralField: 90
Number of cedar.processing.sources.Boost: 23
Number of cedar.processing.ComponentMultiply: 52
Number of cedar.processing.sources.ConstMatrix: 8
Number of cedar.processing.steps.Convolution: 4
Number of cedar.processing.Flip: 4
Number of cedar.processing.sources.GaussInput: 16
Number of cedar.processing.Projection: 38
Number of cedar.processing.sources.SpatialTemplate: 24
Number of cedar.processing.StaticGain: 166


# Get all connections
For the groups the connections to and from the input nodes of the groups and output nodes of the groups have to be handled seperately, since these should be replaced by direct connections between the instances.

The names of the input and output nodes for groups are in the group element "connectors". If they have the value "true" they are input nodes, if the have the value "false" they are output nodes. In connections to input nodes of a group the target name is of the form: group_name.input_node_name

Problems:
- duplicate names in groups: e.g. intention node the name of several neural fields, only in different groups --> cannot use name alone to identify unit when getting rid of groups
- group to group connections: atm able to replace output and input nodes of a group by just removing it as the middle unit between connections to it and from it. BUT if connection from one group output to a group input, then the replaced connection again has a group input as its target --> do sth recursive? test if input/output a group input/output node before replacing the connection

In [7]:
connections_outside_groups = data[key_to_pos_dict["connections"]][1]
len(connections_outside_groups)
connections_outside_groups[2][1]

('target', 'Component Multiply 7.operands')

In [8]:
# get all connectors
connectors = {}
group_connections = {}
for group in groups:
    group_connectors = group[1][4][1]
#     print(group_connectors, "\n")
    connectors[group[0]] = group_connectors
    group_connections[group[0]] = group[1][2][1]

connectors

{'Condition of  Dissatisfaction ': [('CoS Targ Clone Field', 'true'),
  ('Main Behavior', 'false'),
  ('Main Behavior Input', 'true'),
  ('Task True False', 'true')],
 'Match Field': [('CoS Mismatch Field', 'true'),
  ('Intention ', 'true'),
  ('Mismatch field', 'false'),
  ('TargetFieldPrecNodes', 'false'),
  ('Task Input', 'true'),
  ('Wipe CoS', 'true')],
 'OC Field and Spatial Production Nodes  ': [('CoS Above', 'true'),
  ('CoS Below', 'true'),
  ('CoS Find the Reference Task', 'true'),
  ('CoS Find the Target Task', 'true'),
  ('CoS Imagine Task', 'true'),
  ('CoS Left', 'true'),
  ('CoS North-East', 'true'),
  ('CoS North-West', 'true'),
  ('CoS OCF', 'true'),
  ('CoS Right', 'true'),
  ('CoS South-East', 'true'),
  ('CoS South-West', 'true'),
  ('CoS Spatial Memory Nodes', 'true'),
  ('CoS True False Task', 'true'),
  ('CoS Wipe', 'true'),
  ('Main Behavior', 'false'),
  ('Main Behavior Input', 'true'),
  ('OCF', 'false'),
  ('Spatial Rel Nodes', 'false'),
  ('external input', 

In [9]:
test_conn = group_connections['Condition of  Dissatisfaction '][0]

def add_group_name(group_connection, group_name):
    new_group_connection = [('source', group_name + "." + group_connection[0][1]),
                            ('target', group_name + "." + group_connection[1][1])]
    return new_group_connection

    
print(test_conn)
new_test_conn = add_group_name(test_conn, "Condition of Dissatisfaction")
for group in group_connections:
    add_group_name_lambda = lambda x: add_group_name(x, group)
    new_group_connections = list(map(add_group_name_lambda, group_connections[group]))
#     print(new_group_connections, '\n')
    group_connections[group] = new_group_connections

[('source', 'CoS node.sigmoided activation'), ('target', 'new StaticGain 2.input')]


In [10]:
# group_connections

## Group outputs

In [11]:
# get all group output nodes --> in group_outputs
# get all connections that the group output nodes are part of --> in group_output_connections
# divided into input connections to the output node and output connections from the output node

group_outputs = []

for group_name in connectors:
    for connector in connectors[group_name]:
        # true means it's an input connection, false it's an output connection
        if connector[1] == 'false':
            group_outputs.append((group_name, connector[0]))
            
group_output_connections = {}
for group_output in group_outputs:
    source_string = group_output[0] + "." + group_output[1]
    group_output_connections[group_output] = {'output connections': [], 
                                            'input connections': []}
    # go through outside_group connections
    for connection in connections_outside_groups:
        if connection[0][1] == source_string:
            # check if target is a group input
            group_output_connections[group_output]['output connections'].append(connection) 
            connections_outside_groups.remove(connection)
    # go through connections inside the group
    target_string = group_output[0] + "." + group_output[1] + ".input"
    for connection in group_connections[group_output[0]]:
        if connection[1][1] == target_string:
            group_output_connections[group_output]['input connections'].append(connection)
            # remove the connection since it will be substituted by a new one
            group_connections[group_output[0]].remove(connection)

In [12]:
# for group_connection in group_output_connections:
#     if len(group_output_connections[group_connection]['output connections']) > 1:
#         print(group_connection)

## Replace group input and output connections

In [13]:
ex_replace_conn = group_output_connections[list(group_output_connections.keys())[0]]


def replace_middle_connection(middle_connection_dict):
    ''' takes as input a dictionary with input connections to a node and 
        output connections from the node and creates new connections substituting
        the middle node and connecting the inputs from the input nodes and the 
        outputs from the output nodes directly.
    '''
    # each input has to be connected to each output
    connections_list = []

    # go through all inputs
    for inp_connection in middle_connection_dict['input connections']:
        # source at position 0
        source = inp_connection[0]
        # go through all outputs
        for out_connection in middle_connection_dict['output connections']:
            # target at position 1
            target = out_connection[1]
            connections_list.append([source, target])
#     print(connections_list, '\n')
    return connections_list

new_conns = list(map(replace_middle_connection, group_output_connections.values()))
# for conn in new_conns:
#     print(conn)
# print(len(new_conns))
# for lst in new_conns:
#     if len(lst) > 1:
#         print(lst, '\n')

[connections_outside_groups.extend(conns) for conns in new_conns]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [14]:
# connections_outside_groups

## Group inputs

In [15]:
group_inputs = []

for group_name in connectors:
    for connector in connectors[group_name]:
        # true means it's an input connection, false it's an output connection
        if connector[1] == 'true':
            group_inputs.append((group_name, connector[0]))
            
group_inputs
group_input_connections = {}
for group_input in group_inputs:
    target_string = group_input[0] + "." + group_input[1]
    group_input_connections[group_input] = {'input connections': [], 
                                            'output connections': []}
    # go through outside_group connections
    for connection in connections_outside_groups:
        if connection[1][1] == target_string:
            group_input_connections[group_input]['input connections'].append(connection) 
            # remove the connection since it will be substituted by a new one 
            connections_outside_groups.remove(connection)
    # go through connections inside the group
    source_string = group_input[0] + "." + group_input[1] + ".output"
    for connection in group_connections[group_input[0]]:
        if connection[0][1] == source_string:
            group_input_connections[group_input]['output connections'].append(connection)
            # remove the connection since it will be substituted by a new one
            group_connections[group_input[0]].remove(connection)
    

In [16]:
new_conns = new_conns = list(map(replace_middle_connection, group_input_connections.values()))

_ = [connections_outside_groups.extend(conns) for conns in new_conns]
_ = [connections_outside_groups.extend(conns) for conns in group_connections.values()]

In [17]:
connections_outside_groups

[[('source', 'Target Orange Production.sigmoided activation'),
  ('target', 'Component Multiply 8.operands')],
 [('source', 'Gauss Input 7.Gauss input'),
  ('target', 'Component Multiply 9.operands')],
 [('source', 'Gauss Input 8.Gauss input'),
  ('target', 'Component Multiply 7.operands')],
 [('source', 'Target Cyan Production.sigmoided activation'),
  ('target', 'Component Multiply 7.operands')],
 [('source', 'Component Multiply 7.product'), ('target', 'Colour.input')],
 [('source', 'Gauss Input 9.Gauss input'),
  ('target', 'Component Multiply 8.operands')],
 [('source', 'Component Multiply 8.product'), ('target', 'Colour.input')],
 [('source', 'Component Multiply 9.product'), ('target', 'Colour.input')],
 [('source', 'Target Blue Production.sigmoided activation'),
  ('target', 'Component Multiply 9.operands')],
 [('source', 'Perception.sigmoided activation'),
  ('target', 'Attention.input')],
 [('source', 'Colour.sigmoided activation'),
  ('target', 'Static Gain 2.input')],
 [('sou